In [1]:
import cPickle as pkl
import pandas as pd
import numpy as np
import os.path

In [2]:
dirname = '../../results_20180424'

In [3]:
descriptor = '_700_transcripts'

In [10]:
files = !ls {dirname}
files = [os.path.join(dirname, filename) for filename in files]
files

['../../results_20180424/test configuration with 2 genes in 700 transcripts, 100 ribosomes, deltat=0.05s_20180424_0848_100_ribosomes_0200s.p',
 '../../results_20180424/test configuration with 2 genes in 700 transcripts, 10 ribosomes, deltat=0.05s_20180424_0847_10_ribosomes_0200s.p',
 '../../results_20180424/test configuration with 2 genes in 700 transcripts, 1 ribosomes, deltat=0.05s_20180424_0846_1_ribosomes_0200s.p',
 '../../results_20180424/test configuration with 2 genes in 700 transcripts, 200 ribosomes, deltat=0.05s_20180424_0849_200_ribosomes_0200s.p',
 '../../results_20180424/test configuration with 2 genes in 700 transcripts, 20 ribosomes, deltat=0.05s_20180424_0847_20_ribosomes_0200s.p',
 '../../results_20180424/test configuration with 2 genes in 700 transcripts, 2 ribosomes, deltat=0.05s_20180424_0846_2_ribosomes_0200s.p',
 '../../results_20180424/test configuration with 2 genes in 700 transcripts, 30 ribosomes, deltat=0.05s_20180424_0847_30_ribosomes_0200s.p',
 '../../resul

Blacklisted species:

In [11]:
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein', 
             'proteins', 'peptide_bonds']

Data frame at cell level:

In [12]:
dfc = pd.DataFrame(columns=['ribosomes', 'transcripts', 'collisions', 'nocollisions', 'peptide_bonds'])

In [13]:
rows = []
dft_columns = ['length', 'init_rate', 'ribosomes', 'tic_toc', 'geneID', 'transcripts', 'index']

for i, filename in enumerate(files):
    print filename
    results = pkl.load(open(filename))
    
    # append to cell level data frame
    print 'cell level...'
    rowvals = [results['n_ribosomes'], len(results['transcriptome']), results['collisions'], 
               results['nocollisions'], results['peptide_bonds']]
    row = dict(zip(list(dfc.columns), rowvals))
    #print row
    dfc = dfc.append(row, ignore_index=True)
    #print dfc.head()
    
    # append to transcript level data frame
    print 'transcript level...'
    
    for transcript in results['transcriptome']:
        av_time = np.mean([tictoc[1] - tictoc[0] for tictoc in transcript.tic_toc])
        rowvals = [transcript.geneID, transcript.index, transcript.init_rate, transcript.length, 
                   results['n_ribosomes'], av_time, len(results['transcriptome'])]
        row = dict(zip(list(dft_columns), rowvals))
        #print row
        rows.append(row)
            
    # append to gene level data frame
    print 'gene level...'
    header = str(results['n_ribosomes']) + '_' + str(len(results['transcriptome']))
    tc = results['timecourses']
    efficiency_items = [(gene, (tc[gene][-1] - tc[gene][-2001]) / 100.) for gene in tc if gene not in blacklist]
    tmp = pd.DataFrame(efficiency_items, columns = ['gene', header]).set_index('gene')
    
    if i == 0:  # first time
        dfg = tmp
    else:
        dfg = pd.merge(dfg, tmp, left_index=True, right_index=True, how='outer')

dfc.sort_values('ribosomes', inplace=True)

dft = pd.DataFrame(rows)
dft.columns = ['ribosomes', 'transcripts', 'index', 'geneID', 'init_rate', 'length', 'tic_toc']

../../results_20180424/test configuration with 2 genes in 700 transcripts, 100 ribosomes, deltat=0.05s_20180424_0848_100_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180424/test configuration with 2 genes in 700 transcripts, 10 ribosomes, deltat=0.05s_20180424_0847_10_ribosomes_0200s.p


/home/martin/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/martin/.local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell level...
transcript level...
gene level...
../../results_20180424/test configuration with 2 genes in 700 transcripts, 1 ribosomes, deltat=0.05s_20180424_0846_1_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180424/test configuration with 2 genes in 700 transcripts, 200 ribosomes, deltat=0.05s_20180424_0849_200_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180424/test configuration with 2 genes in 700 transcripts, 20 ribosomes, deltat=0.05s_20180424_0847_20_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180424/test configuration with 2 genes in 700 transcripts, 2 ribosomes, deltat=0.05s_20180424_0846_2_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180424/test configuration with 2 genes in 700 transcripts, 30 ribosomes, deltat=0.05s_20180424_0847_30_ribosomes_0200s.p
cell level...
transcript level...
gene level...
../../results_20180424/test c

In [14]:
dfc

,ribosomes,transcripts,collisions,nocollisions,peptide_bonds
2,1.0,700.0,0.0,104.0,1138.0
5,2.0,700.0,1.0,197.0,2197.0
7,3.0,700.0,2.0,300.0,3328.0
9,5.0,700.0,5.0,511.0,5630.0
11,8.0,700.0,10.0,838.0,9316.0
1,10.0,700.0,26.0,1020.0,11420.0
4,20.0,700.0,99.0,2057.0,22843.0
6,30.0,700.0,227.0,3101.0,34455.0
8,50.0,700.0,688.0,5113.0,56883.0
10,80.0,700.0,1793.0,8205.0,91117.0


In [16]:
dft = dft.dropna(subset=['tic_toc'])
dft

,ribosomes,transcripts,index,geneID,init_rate,length,tic_toc
0,100,700,0,1,0.100,36,1.650000
1,100,700,1,1,0.100,36,1.981034
2,100,700,2,1,0.100,36,1.813462
3,100,700,3,1,0.100,36,2.080000
4,100,700,4,1,0.100,36,1.829167
5,100,700,5,1,0.100,36,1.969565
6,100,700,6,1,0.100,36,1.980435
7,100,700,7,1,0.100,36,1.610417
8,100,700,8,1,0.100,36,1.840476
9,100,700,9,1,0.100,36,1.818182


In [17]:
dfg

,100_700,10_700,1_700,200_700,20_700,2_700,30_700,3_700,50_700,5_700,80_700,8_700
gene,,,,,,,,,,,,
1,50.81,4.97,0.51,98.92,10.14,0.97,15.17,1.42,25.21,2.50,40.48,4.19
2,0.41,0.09,NaN,1.33,0.07,0.01,0.17,0.02,0.22,0.01,0.29,0.02
tRNA_free_01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
tRNA_free_07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [18]:
dfc.to_pickle(dirname + '/dfc' + descriptor + '.p')

In [19]:
dft.to_pickle(dirname + '/dft' + descriptor + '.p')

In [20]:
dfg.to_pickle(dirname + '/dfg' + descriptor + '.p')

Here we start the analysis (old notebook 17):

In [21]:
dfc['collisions_percentage'] = dfc['collisions'] / (dfc['collisions'] + dfc['nocollisions'])

In [22]:
dfc

,ribosomes,transcripts,collisions,nocollisions,peptide_bonds,collisions_percentage
2,1.0,700.0,0.0,104.0,1138.0,0.000000
5,2.0,700.0,1.0,197.0,2197.0,0.005051
7,3.0,700.0,2.0,300.0,3328.0,0.006623
9,5.0,700.0,5.0,511.0,5630.0,0.009690
11,8.0,700.0,10.0,838.0,9316.0,0.011792
1,10.0,700.0,26.0,1020.0,11420.0,0.024857
4,20.0,700.0,99.0,2057.0,22843.0,0.045918
6,30.0,700.0,227.0,3101.0,34455.0,0.068209
8,50.0,700.0,688.0,5113.0,56883.0,0.118600
10,80.0,700.0,1793.0,8205.0,91117.0,0.179336


In [23]:
ax = dfc.plot(x='ribosomes', y='collisions_percentage', figsize=(10, 7), fontsize=16)
ax.legend(["5 transcripts"], prop={'size': 16})
#plt.show()

In [24]:
dft_grouped = dft.groupby(['geneID', 'init_rate', 'length', 'ribosomes', 'transcripts'], as_index=False).mean()
dft_grouped

,geneID,init_rate,length,ribosomes,transcripts,index,tic_toc
0,1,0.100,36,1,700,185.057471,1.889080
1,1,0.100,36,2,700,209.927152,1.933499
2,1,0.100,36,3,700,201.981818,1.948902
3,1,0.100,36,5,700,196.517606,1.907113
4,1,0.100,36,8,700,198.950292,1.854956
5,1,0.100,36,10,700,199.059783,1.886413
6,1,0.100,36,20,700,199.615578,1.897341
7,1,0.100,36,30,700,199.500000,1.864579
8,1,0.100,36,50,700,199.500000,1.889542
9,1,0.100,36,80,700,199.500000,1.887779
